In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# hyperparameters
block_size = 256
batch_size = 16
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 50
dropout = 0
learning_rate = 1e-3
n_embd = 512
n_layer = 4
n_head = 4

text = open('../data/shakespeare.txt', encoding='utf-8').read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

# map chars to tokens
itos = { i: ch for i, ch in enumerate(chars) }
stoi = { ch: i for i, ch in enumerate(chars) }
encode = lambda s: [stoi[ch] for ch in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text))
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    X = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    X, y = X.to(device), y.to(device)
    return X, y

@torch.no_grad()
def estimate_loss():
    model.eval()
    out = {}
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, y = get_batch(split)
            logits, loss = model(X, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        B,T,C = x.shape
        
        k = self.key(x)
        q = self.query(x)
        #compute attention scores
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        # value
        v = self.value(x)
        out = wei @ v
        return out
    
class MultiHeadAttention(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_head)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        heads = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(heads)
        out = self.dropout(out)
        return out
    
    
    
class FeedForward(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd),
            nn.Dropout(dropout)
        )
        
    def forward(self, x):
        x = self.net(x)
        return x
    
    
    
class Block(nn.Module):
    def __init__(self):
        super().__init__()
        
        head_size = n_embd // n_head
        
        self.sa = MultiHeadAttention(head_size)
        self.ffwd = FeedForward()
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        x = self.sa(x)
        x = self.ffwd(x)
        x = self.dropout(x)
        return x
    
    
class Transformer(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block() for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, idx, targets=None):
        B,T = idx.shape
        
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(block_size).to(device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        if targets == None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss
    
model = Transformer().to(device)
m = model.to(device)
print(sum(p.numel() for p in m.parameters()) / 1e6, 'M parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)



epochs = 200
for epoch in range(epochs):
    
    if epoch % (epochs / 10) == 0 or epoch == epochs - 1:
        losses = estimate_loss()
        print(f"step {epoch}: train loss {losses['train']:.4f}, val loss {losses['val']}")
        
        
    xb, yb = get_batch('train')
    
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

12.793921 M parameters
step 0: train loss 4.2368, val loss 4.24380350112915
step 20: train loss 3.3261, val loss 3.3619747161865234
step 40: train loss 3.3159, val loss 3.3554022312164307
step 60: train loss 3.3257, val loss 3.3475563526153564
step 80: train loss 3.3098, val loss 3.3524062633514404
step 100: train loss 3.3111, val loss 3.351717948913574
step 120: train loss 3.3218, val loss 3.352501630783081
step 140: train loss 3.3132, val loss 3.367694139480591
step 160: train loss 3.3139, val loss 3.3571321964263916
step 180: train loss 3.3245, val loss 3.354311227798462
step 199: train loss 3.3128, val loss 3.3516080379486084
